# Сегментация медицинских изображений с локалайзера компьютерного томографа

Имортированые библиотеки:

In [1]:
import os
import pickle
import pydicom
import numpy as np
import tensorflow as tf

from tqdm import tqdm
from matplotlib import pyplot as plt
from pydicom.datadict import DicomDictionary, keyword_dict
from pydicom.pixel_data_handlers.numpy_handler import pack_bits

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

## Разметка изображений

Задача разметки данных изображений заключается в том, чтобы отметить на снимке один (или несколько) из пяти отделов человеческого тела. Размечатся изображеня будут для пяти типов исследования для каждого отдела соответственно. 

In [2]:
"""
    Необходимо указать тип иссследования:
     * Определение головы на снимке: head_segmentation
     * Определение шейного отдела на снимке: neck_segmentation
     * Определение грудной клетки на снимке: chest_segmentation
     * Определение брюшной полости на снимке: abdomen_segmentation
     * Определение области малого таза на снимке: pelvis_segmentation
"""
STUDY_TYPE = 'head_segmentation'

BODY_PART_NAME = ''
if STUDY_TYPE == 'head_segmentation':
    BODY_PART_NAME = 'Head'
elif STUDY_TYPE == 'neck_segmentation':
    BODY_PART_NAME = 'Neck'
elif STUDY_TYPE == 'chest_segmentation':
    BODY_PART_NAME = 'Chest'
elif STUDY_TYPE == 'abdomen_segmentation':
    BODY_PART_NAME = 'Abdomen'
elif STUDY_TYPE == 'pelvis_segmentation':
    BODY_PART_NAME = 'Pelvis'

Мной использован следующий подход. 

1. Загружается массив изображений, которые необходимо разметить:

In [3]:
def load_images(path):
    all_img = [pydicom.read_file(path + '/' + i) for i in os.listdir(path)]
    return all_img

2. Получаются значения каждого воксела изображения и стандартизуются:

In [4]:
def get_voxels(img):
    image1 = np.stack(img.pixel_array)
    image1 = image1.astype(np.int16)
    image1[image1 == -2000] = 0
    intercept = img.RescaleIntercept
    slope = img.RescaleSlope
    if slope != 1:
        image1 = slope * image1.astype(np.float64)
        image1 = image1.astype(np.int16)
    image1 += np.int16(intercept)
    return np.array(image1, dtype=np.int16)

def standardize_voxel_values(img):
    mean = np.mean(img)
    std = np.std(img)
    img = img-mean
    img = img/std
    return img


def show_standardize_image(img, title='Standardized image'):
    fig, ax = plt.subplots(1, 1, figsize=[10, 10])
    ax.set_title(title)
    ax.imshow(img, cmap='gray')
    ax.axis('on')
    plt.show()

3. Создается ограничительная рамка с интересующим отделом тела, по введенным противоположным координатам этой рамки:

In [5]:
def create_demo_box(img, point_x_beg, point_y_beg, point_x_end, point_y_end):
    img[point_y_beg, point_x_beg:point_x_end] = 1
    img[point_y_end, point_x_beg:point_x_end] = 1
    img[point_y_beg:point_y_end, point_x_beg] = 1
    img[point_y_beg:point_y_end, point_x_end] = 1
    return img

4. Если данная рамка удовлетворяет нас, то закрепляем результат, создая бинарную рамку, которая будет использованя в виде `overlay array`:

In [6]:
def create_box(img, point_x_beg, point_y_beg, point_x_end, point_y_end):
    bound_box = img
    for i in range(len(bound_box)):
        for j in range(len(bound_box[i])):
            bound_box[i][j] = 0

    bound_box[point_y_beg, point_x_beg:point_x_end] = 1
    bound_box[point_y_end, point_x_beg:point_x_end] = 1
    bound_box[point_y_beg:point_y_end, point_x_beg] = 1
    bound_box[point_y_beg:point_y_end, point_x_end] = 1
    return bound_box

5. Накладываем данную рамку на выбранный снимок:

In [7]:
def add_overlay(img, area, array):
    groups = {
        'Head': 0x6000,
        'Neck': 0x6002,
        'Chest': 0x6004,
        'Abdomen': 0x6006,
        'Pelvis': 0x6008,
    }

    string = area + ': Overlay Rows'

    new_dict_items = {
        (groups[area], 0x0010): ('US', '1', string, '', 'OverlayRows'),
        (groups[area], 0x0011): ('US', '1', area + ": Overlay Columns", '', 'OverlayColumns'),
        (groups[area], 0x0015): ('IS', '1', area + ": Number of Frames in Overlay", '', 'NumberFrames'),
        (groups[area], 0x0022): ('LO', '1', area + ": Overlay Description ", '', 'OverlayDescription'),
        (groups[area], 0x0040): ('CS', '1', area + ": Overlay Type", '', 'OverlayType'),
        (groups[area], 0x0050): ('SS', '2', area + ": Overlay Origin", '', 'OverlayOrigin'),
        (groups[area], 0x0051): ('US', '1', area + ": Image Frame Origin ", '', 'ImageFrameOrigin'),
        (groups[area], 0x0100): ('US', '1', area + ": Overlay Bits Allocated", '', 'OverlayBitsAllocated'),
        (groups[area], 0x0102): ('US', '1', area + ": Overlay Bit Position", '', 'OverlayBitPosition'),
        (groups[area], 0x3000): ('OW', '1', area + ": Overlay Data", '', 'OverlayData'),
    }

    DicomDictionary.update(new_dict_items)
    new_names_dict = dict([(val[4], tag) for tag, val in new_dict_items.items()])
    keyword_dict.update(new_names_dict)
    row_count, col_count = array.shape
    img.OverlayRows = row_count
    img.OverlayColumns = col_count
    img.NumberFrames = 1
    img.OverlayDescription = area
    img.OverlayType = 'G'
    img.OverlayOrigin = [1, 1]
    img.ImageFrameOrigin = 1
    img.OverlayBitsAllocated = 1
    img.OverlayBitPosition = 0
    array_new = np.reshape(array, array.size)
    packed_bytes = pack_bits(array_new)

    if len(packed_bytes) % 2:
        packed_bytes += b'\x00'

    img.OverlayData = packed_bytes
    img[groups[area], 0x3000].VR = 'OW'
    return img

***
### Пример применения описанного выше подхода

Опишем пути для исходных и выходных данных:

In [8]:
data_path = '/home/computer/SCIENTIFIC_WORK/segmentation_medical_images/ct_locator_sample'

In [9]:
output_path_with = 'Data/' + STUDY_TYPE + '/' + BODY_PART_NAME + '/'
output_path_without = 'Data/' + STUDY_TYPE + '/Other/'

In [10]:
images = load_images(data_path)

Введем номер интересующего снимка (пусть для примера это будет первый элемент массива снимков):

In [11]:
print('Enter image number:')
number = int(input())
image = images[number]

Enter image number:
0


Отобразим полученное изображение в дополнительном окне и отметим (наведением курсора в области изображения) противоположные вершины ограничительной рамки для выбранного отдела: 

In [12]:
%matplotlib

image_vox = get_voxels(image)
image_std = standardize_voxel_values(image_vox)
show_standardize_image(image_std)

Using matplotlib backend: Qt5Agg


Последовательно введем координаты даные координаты:

In [13]:
print('Enter the coordinates of the first point of the bounding box:')
beg_point_x = int(input())
beg_point_y = int(input())

Enter the coordinates of the first point of the bounding box:
100
10


In [14]:
print('Enter the coordinates of the end point of the bounding box:')
end_point_x = int(input())
end_point_y = int(input())

Enter the coordinates of the end point of the bounding box:
350
250


Получим и выведем на экран, полученную рамку:

In [15]:
image_st = create_demo_box(image_std, beg_point_x, beg_point_y, end_point_x, end_point_y)
show_standardize_image(image_st)

Убедившись в корректности рамки, создаем ее бинарный аналог:

In [16]:
image_for_box = image_std
bounding_box = create_box(image_for_box, beg_point_x, beg_point_y, end_point_x, end_point_y)

Применим рамку к исходному изображению, аннотируем ее наименованием интересующего отдела и выведем метаданные `.dicom`-файла:

In [17]:
print('Enter the name of the marked object: (Head, Neck, Chest, Abdomen, Pelvis)')
part_name = input()
image = add_overlay(image, part_name, bounding_box)

if part_name == 'Head':
    image.save_as(output_path_with + 'image_' + str(number) + '.dcm')
else:
    image.save_as(output_path_without + 'image_' + str(number) + '.dcm')

print(image)

Enter the name of the marked object: (Head, Neck, Chest, Abdomen, Pelvis)
Head
Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 190
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.2.840.113704.1.111.4580.1489087675.28499
(0002, 0010) Transfer Syntax UID                 UI: JPEG Lossless, Non-Hierarchical (Process 14)
(0002, 0012) Implementation Class UID            UI: 1.2.826.0.1.3680043.2.135.1066.101
(0002, 0013) Implementation Version Name         SH: '1.4.16/OTHER'
-------------------------------------------------
(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'LOCALIZER']
(0008, 0012) Instance Creation Date              DA: '19800101'
(0008, 0013) Instance Creation Time              TM: '55555

***

### Использование скрипта для разметки

Для разметки изображений с помощью предложенного метода, предлагаю воспользоваться скриптом [labeling.py](https://github.com/AlexeyPopov1997/MedicalImagesSegmentation/blob/master/labeling.py).
***

## Создание потоков с данными для обучения

В качестве входных данных для обучения модели классификатора отделов человеческого тела со снимков локалайзера компьютерного томографа дудут использоваться два потока байтов: поток признаков, и поток соответствующих значений целефой функции (наименование определеного отдела тела). Для этого используется алгоритм сериализации объектов `Python` модуля `pickle`.

Для создания байтовых потоков предлагаю воспользоваться скриптом [loading_data.py](https://github.com/AlexeyPopov1997/MedicalImagesSegmentation/blob/master/loading_data.py).

***

## Модель распознования интересующего отдела на снимке на основе сверточной нейронной сети (СНС)

Для начала нейбходимо выбрать тип исследования для поиска интересующего отдела на снимке из предложенного набора:
* head_segmentation
* neck_segmentation
* chest_segmentation
* abdomen_segmentation
* pelvis_segmentation

In [18]:
STUDY_TYPE = str(input())

head_segmentation


Далее загрузить подготовленные потоки для обучения:

In [19]:
stream_in = open('Streams/' + STUDY_TYPE + '/X.pickle', 'rb')
X = pickle.load(stream_in)

stream_in = open('Streams/' + STUDY_TYPE + '/y.pickle', 'rb')
y = pickle.load(stream_in)

X = X/255.0

Ввести оптимальные параметры для обучаемой модели: 

*(Для поиска оптимальных параметров для ваших данных и соответствующего типа исследования предлагаю воспользоваться скриптом* [optimizing_models.py](https://github.com/AlexeyPopov1997/MedicalImagesSegmentation/blob/master/optimizing_models.py) *)*

In [20]:
EPOCHS = 5
DENSE_LAYERS = 0
LAYER_SIZE = 256
CONV_LAYERS = 3
BATCH_SIZE = 32

Модель будет выглядить следующим образом:

In [21]:
model = Sequential()

model.add(Conv2D(LAYER_SIZE, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

for layer in range(CONV_LAYERS - 1):
    model.add(Conv2D(LAYER_SIZE, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

for _ in range(DENSE_LAYERS):
    model.add(Dense(LAYER_SIZE))
    model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X, y, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_split=0.3)

Train on 2 samples, validate on 2 samples
Epoch 1/5
2/2 [==============================] - 3s 2s/sample - loss: 0.6516 - accuracy: 1.0000 - val_loss: 4.3096 - val_accuracy: 0.5000
Epoch 2/5
2/2 [==============================] - 0s 176ms/sample - loss: 0.0024 - accuracy: 1.0000 - val_loss: 11.8746 - val_accuracy: 0.5000
Epoch 3/5
2/2 [==============================] - 0s 168ms/sample - loss: 1.5379e-07 - accuracy: 1.0000 - val_loss: 22.2461 - val_accuracy: 0.5000
Epoch 4/5
2/2 [==============================] - 0s 164ms/sample - loss: 3.1727e-13 - accuracy: 1.0000 - val_loss: 34.5782 - val_accuracy: 0.5000
Epoch 5/5
2/2 [==============================] - 0s 164ms/sample - loss: 5.6451e-20 - accuracy: 1.0000 - val_loss: 48.2162 - val_accuracy: 0.5000


Не буду сохранять данную модель. В репозитории для примера присутствует уже обученная мною модель по определению головы на снимках с локалайзера на приемлемом количестве снимков для проверки подхода воспользуюсь ей.

#### Для создания же новой модели, с описанной выше архитектурой, предлагаю воспользоваться скриптом [cnn_model.py](https://github.com/AlexeyPopov1997/MedicalImagesSegmentation/blob/master/cnn_model.py).

***

## Использование обученной модели

Для обученных моделей пердоставлена отдельная директория [Models](https://github.com/AlexeyPopov1997/MedicalImagesSegmentation/tree/master/Models). 

 В репозиторий уже добавлена обученная модель для определения головы на снимке. Обучена данная модель на 140 снимках тренеровочной выборки и 60 вылидационной. Ее графики точности и функции потерь имеют следующий вид: 
 
 ![Изображение](https://github.com/AlexeyPopov1997/MedicalImagesSegmentation/blob/master/Figures/graph.png?raw=true)

Для ипользования этой или других моделей предлагаю воспользоваться скриптом [prediction.py](https://github.com/AlexeyPopov1997/MedicalImagesSegmentation/blob/master/prediction.py).

 ![Изображение](https://github.com/AlexeyPopov1997/MedicalImagesSegmentation/blob/master/Figures/prediction%20testing.png?raw=true)

***